# Random Forest Regression
Important:I should also try with sales/customer for a specific store

In [42]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['figure.figsize'] = [20.0,8.0]
plt.rcParams['figure.dpi']=80

In [43]:
df = pd.read_csv("dataset/preprocessed_train_PCA.csv")

In [44]:
df.shape

(523021, 48)

### Feature engineering

# RANDOM FOREST REGRESSION

In [45]:
df = df.drop(['NumberOfCustomers'], axis=1)
#df = df.drop(['Region','Events'], axis=1)
# I could leave events

In [46]:
#for now let's drop unusable categorical vars
df = df.drop(['Unnamed: 0','StoreType','AssortmentType','Events'], axis=1)

In [47]:
df.head().T

,0,1,2,3,4
StoreID,1000,1000,1000,1000,1000
Date,01/03/2016,02/03/2016,04/03/2016,05/03/2016,06/03/2016
IsHoliday,0,0,0,0,0
IsOpen,1,1,1,1,0
HasPromotions,0,0,0,0,0
NearestCompetitor,326,326,326,326,326
Region,7,7,7,7,7
NumberOfSales,5676,8111,8300,7154,0
Region_AreaKM2,9643,9643,9643,9643,9643
Region_GDP,17130,17130,17130,17130,17130


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

In [53]:
import modify_dataset as md
train = md.get_fake_train4(df)
test_cheat = md.get_fake_test4(df)
y_train=train['NumberOfSales']
y_test=test_cheat['NumberOfSales']

In [54]:
features = list(train.columns.values)
features.remove('NumberOfSales')
features.remove('StoreID')
#features.remove('D_Year')
features.remove('Date')
features.remove('IsOpen')
features.remove('D_DayOfweek')
features.remove('Min_VisibilitykM')
features.remove('Max_VisibilityKm')
features.remove('AssortmentType_WithFishDept')
features.remove('StoreType_StandardMarket')
features.remove('AssortmentType_General')
features.remove('IsHoliday')
features.remove('CloudCover')
features.remove('AssortmentType_WithNFDept')
features.remove('StoreType_SuperMarket')
print("features")

print(features)

features
['HasPromotions', 'NearestCompetitor', 'Region', 'Region_AreaKM2', 'Region_GDP', 'Region_PopulationK', 'WindDirDegrees', 'D_Day', 'D_DayOfYear', 'D_Month', 'D_Year', 'StoreType_HyperMarket', 'StoreType_ShoppingCenter', 'MeanStoreSales', 'MeanRegionSales', 'D_DayOfWeek_cos', 'D_DayOfWeek_sin', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12']


In [55]:
params = range(20, 101, 20)  #50,201,50

In [56]:
scores, tst_scr = validation_curve(RandomForestRegressor(), train[features][:20000],\
                                   y_train[:20000], 'n_estimators', params, \
                                   cv=5, scoring='r2',n_jobs=-1, verbose=2)

[CV] n_estimators=20 .................................................
[CV] n_estimators=40 .................................................
[CV] n_estimators=60 .................................................
[CV] n_estimators=80 .................................................
[CV] .................................. n_estimators=20, total=   6.1s
[CV] n_estimators=100 ................................................
[CV] .................................. n_estimators=40, total=  12.7s
[CV] n_estimators=20 .................................................
[CV] .................................. n_estimators=60, total=  19.4s
[CV] n_estimators=40 .................................................
[CV] .................................. n_estimators=20, total=   6.9s
[CV] n_estimators=60 .................................................
[CV] .................................. n_estimators=80, total=  25.6s
[CV] n_estimators=80 .................................................
[CV] .

KeyboardInterrupt: 

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
params = range(1, 20,2)

In [ ]:
scores, tst_scr = validation_curve(RandomForestRegressor(n_estimators=10), train[features][:20000], \
                                   y_train[:20000], 'max_features', params, \
                                   cv=3, scoring='r2',n_jobs=-1, verbose=2)

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
params = range(5, 101, 5)

In [ ]:
scores, tst_scr = validation_curve(RandomForestRegressor(n_estimators=100), train[features][:20000], \
                                   y_train[:20000], 'max_depth', params, \
                                   cv=3, scoring='r2',n_jobs=-1, verbose=2)

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df.drop('NumberOfSales', axis=1).values, df.NumberOfSales.values,\
 #                                                   test_size = 0.2)
df = df[df.IsOpen == 1]

In [ ]:
train.shape

### Selecting predictive features 

In [ ]:
train[features].head().T

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model1 = RandomForestRegressor(n_estimators=50, max_depth=20, n_jobs=-1, verbose=2)

In [ ]:
# Can I choose to give it numpy arrays or pandas df or I can only pass np arrays?
model1.fit(train[features], train['NumberOfSales'])

In [ ]:
idx = model1.feature_importances_.argsort()[::-1]

In [ ]:
ax = sns.barplot(x=df[features].columns[idx], y=model1.feature_importances_[idx])
_ = plt.setp(ax.get_xticklabels(), rotation=-90)

## Predict

In [ ]:
#saving cheat test to test with perfect lags later
test_cheat = md.get_fake_test4(df)

In [ ]:
test_cheat.head().T

In [ ]:
test_cheat['_NumberOfSales'] = model1.predict(test_cheat[features])

## BIP Error day by day test

In [ ]:
from BIP import get_BIP_error
error1 = get_BIP_error(test_cheat)

## BIP Error cheat test

In [ ]:
from BIP import get_BIP_error
error1 = get_BIP_error(test_cheat)

In [ ]:
# calculationg R2

# calculate RSS
df_RSS = test_cheat
print(df_RSS.shape)

df_RSS['sqrd_diff'] = df_RSS.apply(lambda x: (x['NumberOfSales'] - x['_NumberOfSales'])**2, axis=1)

RSS = df_RSS['sqrd_diff'].sum()
print('RSS:', RSS)
df_RSS.head()

In [ ]:
# calculate TSS

df_TSS = test_cheat
NS_mean = df_TSS['NumberOfSales'].mean()
print('NS_mean', NS_mean)

df_TSS['sqrd_diff'] = df_TSS.apply(lambda x:  (x['_NumberOfSales'] - NS_mean)**2, axis=1)
TSS = df_TSS['sqrd_diff'].sum()
print('TSS:', TSS)
df_TSS.head()

In [ ]:
# calculate R2
print("RSS:", RSS, "   TSS:", TSS)

R2 = 1 - (RSS / TSS)
print('R_2:', R2)


### Saving TEST

In [ ]:
#test.to_csv('./dataset/RFR5_fake_test.csv', index=False)

In [ ]:
estim = model1.estimators_ 

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
mean_absolute_error(test['NumberOfSales'], test['_NumberOfSales'])


In [ ]:
r2_score(test['NumberOfSales'], test['_NumberOfSales'])

In [ ]:
#defining mape
from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true = y_true.reshape(1, -1)
    y_pred = y_pred.reshape(1, -1)
    #y_true, y_pred = check_array(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(test['NumberOfSales'], test['_NumberOfSales'])

In [ ]:
X_train

In [ ]:
X_test[:, [3]].shape

In [ ]:
print("Residual sum of squares: %.2f"
  % ((y_pred - y_test) ** 2).sum())

#why memory error??

In [ ]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w


def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def rmspe_xg(yhat, y):
   # y = y.values
   # y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [ ]:
rmspe_xg(y_pred,y_test)

## Sample of predicted results

In [ ]:
import matplotlib.pyplot as plt

y_pred_subset = test['_NumberOfSales'][500:600]
x_coordinate = [ 1 * i for i in range(len(y_pred_subset)) ]
plt.plot(x_coordinate,y_pred_subset)
plt.show()

## Sample of real results

In [ ]:
#REAL
import matplotlib.pyplot as plt

y_train_subset = test['NumberOfSales'][500:600]
x_coordinate = [ 1 * i for i in range(len(y_train_subset)) ]
plt.plot(x_coordinate,y_train_subset)
plt.show()

## Real-Predicted

In [ ]:
plt.plot(x_coordinate,y_train_subset)
plt.plot(x_coordinate,y_pred_subset)

plt.legend(['Predicted', 'Real'], loc='upper left')
plt.show()

### Saving model

In [ ]:
#import pickle
#RFR_1 = pickle.dumps(model1)
#clf2 = pickle.loads(s)
#clf2.predict(X[0:1])
#pickle.dump(model1, open("models\RFR_1","wb"))
#model1 = pickle.load(file=open("models\RFR_1","rb"))

## For the submission

In [ ]:
df_test = pd.read_csv

In [ ]:
df_test.IsOpen.unique()

In [ ]:
ind_open = df_test.IsOpen == 1
ind_closed = df_test.IsOpen == 0
df_test2 = df_test[ind_open]

In [ ]:
df_test2.drop('IsOpen', axis=1, inplace=True)

In [ ]:
df_test2.head().T

In [ ]:
X_out = df_test2.values

In [ ]:
y_out = model1.predict(X_out)

In [ ]:
df_out = pd.DataFrame(np.zeros(len(df_test)), columns=['NumberOfSales'])

In [ ]:
df_out[ind_open] = y_out.reshape(-1,1)

In [ ]:
df_out.set_index(np.arange(len(df_out))+1, inplace=True)

In [ ]:
df_out.index.name = 'Id'

In [ ]:
df_out.head()